In [1]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold

from src.data.load_data import get_dataset
from src.data.split_data import train_val_split

from src.models.cross_stitch import get_cross_stitch_network_basic

# os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'

# Set the seed value for experiment reproducibility.
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# tf.keras.mixed_precision.set_global_policy('mixed_float16')

## Load  dataset

In [3]:
DATASET_PATH = '../data/processed/mel_spec/Indian Regional Music/'

languages = ['Assamese', 'Bengali', 'Gujarati', 'Hindi', 'Kannada', 'Kashmiri', 'Khasi and Jaintia', 'Konkani', 'Malayalam',
             'Manipuri', 'Marathi', 'Nagamese', 'Nepali', 'Oriya', 'Punjabi', 'Tamil', 'Telegu']

artists = ['Bhupen Hazarika', 'Dipali Barthakur', 'Tarali Sarma', 'Zubeen Garg', 'Arijit Singh', 'Geeta Dutt', 'Manna Dey',
           'Shreya Ghoshal', 'Atul Purohit', 'Devang Patel', 'Falguni pathak', 'Geeta Rabari', 'Alka yagnik', 'Kishore Kumar',
           'Lata Mangeshkar', 'Sonu Nigam', 'Anuradha Bhat', 'B. K. Sumitra', 'P. B. Sreenivas', 'Rajkumar', 'Raj Begum',
           'Rashid Jahangir', 'Shameema Dev Azad', 'Waheed Jeelani', 'George Shadap', 'Rida Gatphoh', 'Soulmate',
           'Wanshankupar Suchiang', 'Amit Naik', 'Lorna Cordeiro', 'Remo Fernandes', 'Xavier Gomes', 'K J Yesudas', 'Mohanlal',
           'Shweta Mohan', 'Sujatha Mohan', 'Kunjabihari', 'Nongmaithem Pahari', 'Pinky Saikhom', 'Pushparani', 
           'Hridaynath Mangeshkar', 'Milind Ingle', 'Shridhar Phadke', 'Vaishali Samant', 'David Konyak', 'Mengu Suokhrie',
           'Silas Kikon', 'Tetseo Sisters', 'Anju Panta', 'Aruna Lama', 'Narayan Gopal', 'Raju Lama', 'Akshay Mohanty',
           'Bishnu Mohan Kabi', 'Ira Mohanty', 'Tapu Mishra', 'Gurdas Maan', 'Hans Raaj Hans', 'Jasmine Sandlas',
           'Jaswinder Brar', 'Hariharan', 'K. S. Chithra', 'S. P. Balasubrahmanyam', 'Sistla Janaki', 'LV Revanth', 'Malavika',
           'S S Thaman', 'S. P. Sailaja']

In [4]:
dataset = {
        'mel_spec': [],
        'language': [],
        'location': [],
        'artist': [],
        'gender': [],
        'song': [],
        'location_id': [],
        'artist_id': [],
        'gender_id': [],
        # 'veteran': [],
        'no_of_artists': [],
        'song_id': [],
        'local_song_id': []
}

In [5]:
dataset = get_dataset(DATASET_PATH, dataset)

In [9]:
train, val, test = train_val_split(dataset, train_size_percentage=0.8, val_size_percentage=0.2)

1638.0 409.0


TypeError: slice indices must be integers or None or have an __index__ method

In [6]:
avg_loss = []
avg_acc = []
avg_recall = []
avg_precision = []
avg_specificity = []
avg_f1 = []

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices((train))
val_ds = tf.data.Dataset.from_tensor_slices((val))
test_ds = tf.data.Dataset.from_tensor_slices((test))

In [7]:
for train_index, test_index in StratifiedKFold(n_splits=10).split(dataset['mel_spec'], dataset['location_id']):
    x_train, y_train_1, y_train_2, y_train_3, y_train_4 = dataset['mel_spec'][train_index], dataset['location_id'][train_index], dataset['artist_id'][train_index], dataset['gender_id'][train_index], dataset['no_of_artists'][train_index]

    x_test, y_test_1, y_test_2, y_test_3, y_test_4 = dataset['mel_spec'][test_index], dataset['location_id'][test_index], dataset['artist_id'][test_index], dataset['gender_id'][test_index], dataset['no_of_artists'][test_index]

    model = get_cross_stitch_network_basic((128, 130))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss={'output_1': tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                    'output_2': tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                    'output_3': tf.keras.losses.BinaryCrossentropy(from_logits=True),
                    'output_4': tf.keras.losses.BinaryCrossentropy(from_logits=True),
                    },
              metrics=['accuracy'])
    #
    history = model.fit(x=x_train, y=(y_train_1, y_train_2, y_train_3, y_train_4), epochs=1, verbose=0)
    acc = model.evaluate(x=x_test, y=(y_test_1, y_test_2, y_test_3, y_test_4))
    # # avg_loss.append(loss)
    # avg_acc.append(acc)
    # avg_recall.append(rec)
    # avg_precision.append(pre)
    # avg_specificity.append(spec)
    # avg_f1.append(f1_score)

2022-06-20 16:16:44.415252: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-20 16:16:44.969006: I tensorflow/core/common_runtime/gpu/gpu_process_state.cc:214] Using CUDA malloc Async allocator for GPU: 0
2022-06-20 16:16:44.969158: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 12535 MB memory:  -> device: 0, name: Quadro RTX 5000, pci bus id: 0000:65:00.0, compute capability: 7.5
2022-06-20 16:16:45.100886: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 12265543680 exceeds 10% of free system memory.
2022-06-20 16:16:53.144532: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 12265543680 exceeds 

ValueError: in user code:

    File "/home/ys/.local/lib/python3.8/site-packages/keras/engine/training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "/home/ys/.local/lib/python3.8/site-packages/keras/engine/training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/ys/.local/lib/python3.8/site-packages/keras/engine/training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "/home/ys/.local/lib/python3.8/site-packages/keras/engine/training.py", line 860, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/home/ys/.local/lib/python3.8/site-packages/keras/engine/training.py", line 918, in compute_loss
        return self.compiled_loss(
    File "/home/ys/.local/lib/python3.8/site-packages/keras/engine/compile_utils.py", line 184, in __call__
        self.build(y_pred)
    File "/home/ys/.local/lib/python3

## Evaluate the model performance

Run the model on the test set and check the model's performance:

In [ ]:
metrics = history.history

plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)

plt.plot(history.epoch, metrics['loss'])
plt.plot(history.epoch, metrics['output1_loss'])
plt.plot(history.epoch, metrics['output2_loss'])
plt.plot(history.epoch, metrics['output3_loss'])
plt.plot(history.epoch, metrics['output4_loss'])
plt.plot(history.epoch, metrics['output5_loss'])
plt.plot(history.epoch, metrics['val_output1_loss'])
plt.plot(history.epoch, metrics['val_output2_loss'])
plt.plot(history.epoch, metrics['val_output3_loss'])
plt.plot(history.epoch, metrics['val_output4_loss'])
plt.plot(history.epoch, metrics['val_output5_loss'])


plt.grid()
plt.xlabel('Epochs')
plt.ylabel('Loss')


plt.subplot(1, 2, 2)
plt.plot(history.epoch, metrics['output1_accuracy'])
plt.plot(history.epoch, metrics['output2_accuracy'])
plt.plot(history.epoch, metrics['output3_accuracy'])
plt.plot(history.epoch, metrics['output4_accuracy'])
plt.plot(history.epoch, metrics['output5_accuracy'])
plt.plot(history.epoch, metrics['val_output1_accuracy'])
plt.plot(history.epoch, metrics['val_output2_accuracy'])
plt.plot(history.epoch, metrics['val_output3_accuracy'])
plt.plot(history.epoch, metrics['val_output4_accuracy'])
plt.plot(history.epoch, metrics['val_output5_accuracy'])

plt.grid()
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.show()

In [ ]:
hist = pd.DataFrame(history.history)
hist[['loss', 'output1_loss', 'output2_loss', 'output3_loss', 'output4_loss',
       'output5_loss', 'val_loss', 'val_output1_loss', 'val_output2_loss', 'val_output3_loss',
       'val_output4_loss', 'val_output5_loss',]].plot(figsize=(20, 10))
plt.legend(loc=9, ncol=8)
plt.grid()
plt.show()

In [ ]:
hist[['output1_accuracy', 'output2_accuracy', 'output3_accuracy', 'output4_accuracy',
      'output5_accuracy','val_output2_accuracy', 'val_output3_accuracy', 'val_output4_accuracy',
       'val_output5_accuracy']].plot(figsize=(20, 10))
plt.legend(loc=9, ncol=7)
plt.grid()
plt.show()

In [ ]:
model.evaluate(test_ds)

In [ ]:
test_label1 = np.concatenate([y1 for mel_spec, (y1, y2, y3, y4, y5) in test_ds], axis=0)
test_label2 = np.concatenate([y2 for mel_spec, (y1, y2, y3, y4, y5) in test_ds], axis=0)
test_label3 = np.concatenate([y3 for mel_spec, (y1, y2, y3, y4, y5) in test_ds], axis=0)
test_label4 = np.concatenate([y4 for mel_spec, (y1, y2, y3, y4, y5) in test_ds], axis=0)
test_label5 = np.concatenate([y5 for mel_spec, (y1, y2, y3, y4, y5) in test_ds], axis=0)

In [ ]:
y_pred1, y_pred2, y_pred3, y_pred4, y_pred5 = model.predict(test_ds, verbose=1)

In [ ]:
y_true1 = test_label1
y_true2 = test_label2
y_true3 = test_label3
y_true4 = test_label4
y_true5 = test_label5

y_pred1 = np.argmax(y_pred1, axis = -1)
y_pred2 = np.argmax(y_pred2, axis = -1)
y_pred3 = np.argmax(y_pred3, axis = -1)
y_pred4 = np.argmax(y_pred4, axis = -1)
y_pred5 = np.argmax(y_pred5, axis = -1)

test_acc1 = sum(y_pred1 == y_true1) / len(y_true1)
print(f'Test set accuracy for label1: {test_acc1:.0%}')

test_acc2 = sum(y_pred2 == y_true2) / len(y_true2)
print(f'Test set accuracy for label2: {test_acc2:.0%}')

test_acc3 = sum(y_pred3 == y_true3) / len(y_true3)
print(f'Test set accuracy for label3: {test_acc3:.0%}')

test_acc4 = sum(y_pred4 == y_true4) / len(y_true4)
print(f'Test set accuracy for label4: {test_acc4:.0%}')

test_acc5 = sum(y_pred5 == y_true5) / len(y_true5)
print(f'Test set accuracy for label5: {test_acc5:.0%}')

### Display a confusion matrix

Use a <a href="https://developers.google.com/machine-learning/glossary#confusion-matrix" class="external">confusion matrix</a> to check how well the model did classifying each of the langs in the test set:


In [ ]:
confusion_mtx = tf.math.confusion_matrix(y_true1, y_pred1)
plt.figure(figsize=(10, 8))
sns.heatmap(confusion_mtx,
            # xticklabels=languages,
            # yticklabels=languages,
            annot=True, fmt='g')
plt.xlabel('Prediction')
plt.ylabel('Label1')
plt.show()

In [ ]:
confusion_mtx = tf.math.confusion_matrix(y_true2, y_pred2)
plt.figure(figsize=(25, 15))
sns.heatmap(confusion_mtx,
            # xticklabels=artists,
            # yticklabels=artists,
            annot=True, fmt='g')
plt.xlabel('Prediction')
plt.ylabel('Label2')
plt.show()

In [ ]:
confusion_mtx = tf.math.confusion_matrix(y_true3, y_pred3)
plt.figure(figsize=(10, 8))
sns.heatmap(confusion_mtx,
            # xticklabels=artists,
            # yticklabels=artists,
            annot=True, fmt='g')
plt.xlabel('Prediction')
plt.ylabel('Label3')
plt.show()

In [ ]:
confusion_mtx = tf.math.confusion_matrix(y_true5, y_pred5)
plt.figure(figsize=(15, 10))
sns.heatmap(confusion_mtx,
            # xticklabels=artists,
            # yticklabels=artists,
            annot=True, fmt='g')
plt.xlabel('Prediction')
plt.ylabel('Label5')
plt.show()